In [18]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import pandas as pd
import time

In [81]:
city_name = "Paris"
delay = 5 # seconds

links = {}

In [82]:
driver = webdriver.Chrome("C:\\Users\\wirze\\DataspellProjects\\chromedriver_win32\\chromedriver.exe")
driver.get("https://www.tripadvisor.com/Search?q=parks&searchSessionId=CF4A26D0C451E0783853C96C46AC8B131637532036921ssid&sid=72ED914F7F2B4263B4C7FB9FF61A5A671637532046511&geo=186338&blockRedirect=true&ssrc=a")
javaScript = "PagerNextClick();"

inputElement = driver.find_element_by_id("GEO_SCOPED_SEARCH_INPUT")
inputElement.send_keys('Moscow')
time.sleep(2)
clearElement = driver.find_element_by_id("CLEAR_WHERE")
clearElement.click()
time.sleep(2)
inputElement = driver.find_element_by_id("GEO_SCOPED_SEARCH_INPUT")
inputElement.send_keys(city_name)
time.sleep(2)
inputElement.send_keys(Keys.ENTER)
time.sleep(3)
parksElement = driver.find_elements_by_xpath("//*[contains(text(), 'Parks in')]")
parksElement[1].click()

C:\Users\wirze\AppData\Local\Temp/ipykernel_17200/463546191.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\\Users\\wirze\\DataspellProjects\\chromedriver_win32\\chromedriver.exe")
C:\Users\wirze\AppData\Local\Temp/ipykernel_17200/463546191.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  inputElement = driver.find_element_by_id("GEO_SCOPED_SEARCH_INPUT")
C:\Users\wirze\AppData\Local\Temp/ipykernel_17200/463546191.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  clearElement = driver.find_element_by_id("CLEAR_WHERE")
C:\Users\wirze\AppData\Local\Temp/ipykernel_17200/463546191.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  inputElement = driver.find_element_by_id("GEO_SCOPED_SEARCH_INPUT")
C:\Users\wirze\AppData\Local\Temp/ipykernel_17200/46

In [83]:
for i in range(42):
    html = driver.page_source
    soup = BeautifulSoup(html)

    results_names = soup.find_all("div", class_="csemS")
    results_links = soup.find_all("div", class_="fVbwn cdAAV cagLQ eZTON dofsx")
    del results_links[1::2]  # delete every second element (counting from the first)

    for result_link, result_name in zip(results_links, results_names):
        if result_link.find("a") is not None:
            links[result_name.text] = result_link.find("a")['href']
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'csemS')))
        NextPageElement = driver.find_elements_by_css_selector('[aria-label="Next page"]')
        print ("Page is ready!")
    except TimeoutException:
        print ("Loading took too much time!")
        time.sleep(5)

    if len(NextPageElement) == 0:
        break
    NextPageElement[0].click()
    time.sleep(5)

C:\Users\wirze\AppData\Local\Temp/ipykernel_17200/2944505539.py:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  NextPageElement = driver.find_elements_by_css_selector('[aria-label="Next page"]')


Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!


In [84]:
list(links)[-1]

'171. Jardin de la place Louise Losserand'

In [85]:
df_links_all =  pd.read_csv('all_parks_links.csv')

df_links = pd.DataFrame.from_dict(links,  orient='index', columns=["link"]).reset_index()
df_links['city'] = city_name
df_links = df_links[['city', 'index', 'link']]

df_links_all2 = pd.concat([df_links_all, df_links], ignore_index=True)
df_links_all2 = df_links_all2[['city', 'index', 'link']]
df_links_all2.to_csv("all_parks_links.csv", index=False)

In [86]:
df_links_all2

,city,index,link
0,test_city,test_park,test_link
1,Saint-Petersburg,1. Catherine Palace and Park,/Attraction_Review-g811323-d301025-Reviews-Cat...
2,Saint-Petersburg,31. Inchkhona Skver,/Attraction_Review-g2394587-d18344144-Reviews-...
3,Saint-Petersburg,32. Udelny Park of Recreation,/Attraction_Review-g298507-d6502804-Reviews-Ud...
4,Saint-Petersburg,33. Ekateringof Recreation Park,/Attraction_Review-g298507-d6502916-Reviews-Ek...
...,...,...,...
520,Paris,167. Square Cyprian Norwid,/Attraction_Review-g187147-d23007212-Reviews-S...
521,Paris,168. Square Jules Durand,/Attraction_Review-g187147-d23035203-Reviews-S...
522,Paris,169. Square Alésia Ridder,/Attraction_Review-g187147-d23035214-Reviews-S...
523,Paris,170. Square Frédéric Bazille,/Attraction_Review-g187147-d21025430-Reviews-S...


In [87]:
df_links_all =  pd.read_csv('all_parks_links.csv')
df_links_all

,city,index,link
0,test_city,test_park,test_link
1,Saint-Petersburg,1. Catherine Palace and Park,/Attraction_Review-g811323-d301025-Reviews-Cat...
2,Saint-Petersburg,31. Inchkhona Skver,/Attraction_Review-g2394587-d18344144-Reviews-...
3,Saint-Petersburg,32. Udelny Park of Recreation,/Attraction_Review-g298507-d6502804-Reviews-Ud...
4,Saint-Petersburg,33. Ekateringof Recreation Park,/Attraction_Review-g298507-d6502916-Reviews-Ek...
...,...,...,...
520,Paris,167. Square Cyprian Norwid,/Attraction_Review-g187147-d23007212-Reviews-S...
521,Paris,168. Square Jules Durand,/Attraction_Review-g187147-d23035203-Reviews-S...
522,Paris,169. Square Alésia Ridder,/Attraction_Review-g187147-d23035214-Reviews-S...
523,Paris,170. Square Frédéric Bazille,/Attraction_Review-g187147-d21025430-Reviews-S...


In [61]:
driver.quit()

In [ ]:
n_pages = 1500
for i in range(n_pages):
    try:
        driver.execute_script(javaScript)
        # driver.implicitly_wait(3) # seconds

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        results = soup.find_all("a", class_="appeal-title-link")
        for result in results:
            links[re.findall(r'\d+', result['href'])[0]] = result['href']

    finally:
        print(i/n_pages)

driver.quit()

In [130]:
links_messages = {}
links_headings = {}

In [131]:
for link in links:
    URL = "http://zpp.rospotrebnadzor.ru"+links[link]
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    page_message = soup.find("p", class_="appeal-details-message")
    page_heading = soup.find("h2")

    links_messages[link] = page_message.text
    links_headings[link] = page_heading.text

In [132]:
len(links_messages)

2150

In [133]:
df_links = pd.DataFrame.from_dict(links,  orient='index', columns=["link"]).reset_index()
df_links

,index,link
0,30056,/Forum/Appeals/Details/30056
1,30047,/Forum/Appeals/Details/30047
2,30043,/Forum/Appeals/Details/30043
3,30037,/Forum/Appeals/Details/30037
4,30016,/Forum/Appeals/Details/30016
...,...,...
2145,26658,/Forum/Appeals/Details/26658
2146,26654,/Forum/Appeals/Details/26654
2147,26652,/Forum/Appeals/Details/26652
2148,26651,/Forum/Appeals/Details/26651


In [134]:
df_messages = pd.DataFrame.from_dict(links_messages,  orient='index', columns=["message"]).reset_index()
df_messages

,index,message
0,30056,"Прошу предоставить правовую информацию, на осн..."
1,30047,Здравствуйте! По договору розничной купли-про...
2,30043,"здравствуйте, я Шагеева Ольга Владимировна, пр..."
3,30037,Здравствуйте! В сети интернет увидела введение...
4,30016,"Здравствуйте! Пожалуйста, примите меры ! https..."
...,...,...
2145,26658,"Как и куда отправить ответ ПЦР-теста, если нет..."
2146,26654,Здравствуйте! Сайт Zengram списали с моей карт...
2147,26652,Прошу помочь урегулировать вопрос по страховым...
2148,26651,"Добрый день! Сегодня обнаружила, что вчера веч..."


In [135]:
df_headings = pd.DataFrame.from_dict(links_headings,  orient='index', columns=["heading"]).reset_index()
df_headings

,index,heading
0,30056,Заявление-разьяснение
1,30047,Уклонение от исполнения договора
2,30043,торговля подделкой красной икры
3,30037,Вопрос о втором законопроекте
4,30016,Мошенники
...,...,...
2145,26658,Как отправить ПЦР тест?
2146,26654,Отказали в возврате денежных средств за неиспо...
2147,26652,Мошеннические схемы ГАЗЭНЕРГОБАНКа
2148,26651,Сайт отказал в возврате средств за неиспользов...


In [136]:
df = df_links.merge(df_headings).merge(df_messages)
df.to_csv("small_data.csv")
df

,index,link,heading,message
0,30056,/Forum/Appeals/Details/30056,Заявление-разьяснение,"Прошу предоставить правовую информацию, на осн..."
1,30047,/Forum/Appeals/Details/30047,Уклонение от исполнения договора,Здравствуйте! По договору розничной купли-про...
2,30043,/Forum/Appeals/Details/30043,торговля подделкой красной икры,"здравствуйте, я Шагеева Ольга Владимировна, пр..."
3,30037,/Forum/Appeals/Details/30037,Вопрос о втором законопроекте,Здравствуйте! В сети интернет увидела введение...
4,30016,/Forum/Appeals/Details/30016,Мошенники,"Здравствуйте! Пожалуйста, примите меры ! https..."
...,...,...,...,...
2145,26658,/Forum/Appeals/Details/26658,Как отправить ПЦР тест?,"Как и куда отправить ответ ПЦР-теста, если нет..."
2146,26654,/Forum/Appeals/Details/26654,Отказали в возврате денежных средств за неиспо...,Здравствуйте! Сайт Zengram списали с моей карт...
2147,26652,/Forum/Appeals/Details/26652,Мошеннические схемы ГАЗЭНЕРГОБАНКа,Прошу помочь урегулировать вопрос по страховым...
2148,26651,/Forum/Appeals/Details/26651,Сайт отказал в возврате средств за неиспользов...,"Добрый день! Сегодня обнаружила, что вчера веч..."
